In [50]:
from analysis.data import Data
import pickle

import numpy as np
import random

def kMedoids(D, k, tmax=100):
    # determine dimensions of distance matrix D
    m, n = D.shape

    if k > n:
        raise Exception('too many medoids')

    # find a set of valid initial cluster medoid indices since we
    # can't seed different clusters with two points at the same location
    valid_medoid_inds = set(range(n))
    invalid_medoid_inds = set([])
    rs,cs = np.where(D==0)
    # the rows, cols must be shuffled because we will keep the first duplicate below
    index_shuf = list(range(len(rs)))
    np.random.shuffle(index_shuf)
    rs = rs[index_shuf]
    cs = cs[index_shuf]
    for r,c in zip(rs,cs):
        # if there are two points with a distance of 0...
        # keep the first one for cluster init
        if r < c and r not in invalid_medoid_inds:
            invalid_medoid_inds.add(c)
    valid_medoid_inds = list(valid_medoid_inds - invalid_medoid_inds)

    if k > len(valid_medoid_inds):
        raise Exception('too many medoids (after removing {} duplicate points)'.format(
            len(invalid_medoid_inds)))

    # randomly initialize an array of k medoid indices
    M = np.array(valid_medoid_inds)
    np.random.shuffle(M)
    M = np.sort(M[:k])

    # create a copy of the array of medoid indices
    Mnew = np.copy(M)

    # initialize a dictionary to represent clusters
    C = {}
    for t in range(tmax):
        # determine clusters, i. e. arrays of data indices
        J = np.argmin(D[:,M], axis=1)
        for kappa in range(k):
            C[kappa] = np.where(J==kappa)[0]
        # update cluster medoids
        for kappa in range(k):
            J = np.mean(D[np.ix_(C[kappa],C[kappa])],axis=1)
            j = np.argmin(J)
            Mnew[kappa] = C[kappa][j]
        np.sort(Mnew)
        # check for convergence
        if np.array_equal(M, Mnew):
            break
        M = np.copy(Mnew)
    else:
        # final update of cluster memberships
        J = np.argmin(D[:,M], axis=1)
        for kappa in range(k):
            C[kappa] = np.where(J==kappa)[0]
 
    #Dooie kut return omschrijven
    new_dict = {}
    for key, value in C.items():
        for element in value:
            new_dict[element] = key

    new_list = [None] * len(new_dict)
    for key, value in new_dict.items():
        new_list[key] = value

    labels = np.array(new_list)
    # return results
    return M, labels

In [51]:
from sklearn.datasets import load_iris
from scipy.spatial import distance_matrix
from sklearn.metrics import silhouette_score
import pandas as pd

data = load_iris().get('data')
target = load_iris().get('target')
dm = distance_matrix(data, data)

In [57]:
for i in range(2,6):
    m, c = kMedoids(dm, i)
    sil_score = silhouette_score(data, c)
    print('sil score {}: '.format(i), sil_score)

sil score 2:  0.685788171261719
sil score 3:  0.5238572613791126
sil score 4:  0.5032512948750133
sil score 5:  0.4278975436200692
